In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.signal
from scipy import *
import copy

---
# VIDEO: Mean-smooth a time series
---

In [ ]:
srate = 1000
time = np.arange(0,3,1/srate)
n = len(time)
p = 15 # poles for random interpolation

# noise level, measured in standard deviations
noiseamp = 5

# amplitude modulator and noise level
ampl = np.interp(np.linspace(0,p,n), np.arange(0,p), np.random.rand(p)*30)
noise = noiseamp * np.random.randn(n)
signal = ampl + noise

plt.plot(time, ampl)
plt.show()
plt.plot(time, noise)
plt.show()
plt.plot(time, signal)
plt.show()